<a href="https://colab.research.google.com/github/dhruv369/AI/blob/master/EVA4_Session_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #1 x 28 x 28 -? 32 x 28 x28 ? 3
        self.bnorm1 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(2, 2) # 32 x 14 x 14 ? 6

        self.conv2 = nn.Conv2d(32, 32, 3, padding=1) #32 x 14 x 14 -? 32 x 14 x14 ? 8
        self.bnorm2 = nn.BatchNorm2d(32)
        self.pool2 = nn.MaxPool2d(2, 2) # 32 x 7 x 7

        self.conv3 = nn.Conv2d(32, 32, 3) #32 x 7 x 7 -? 32 x 5 x 5 ? 
        self.bnorm3 = nn.BatchNorm2d(32)
        # self.pool3 = nn.MaxPool2d(2, 2) #32 x 26 x 26 -? 32 x 24 x24 ? 14
        self.drop1 = nn.Dropout(0.25)

        # self.conv4 = nn.Conv2d(32, 32, 3) #32 x 12 x 12 ? 32 x 10 x 10 ? 16
        # self.bnorm4 = nn.BatchNorm2d(32)
        # self.pool2 = nn.MaxPool2d(2, 2) #32 x 10 x 10 ? 32 x 5 x 5 ? 18

        # self.conv5 = nn.Conv2d(64, 64, 3,padding=1)
        # self.bnorm5 = nn.BatchNorm2d(64)
        # self.pool3 = nn.MaxPool2d(2, 2)
        # self.drop2 = nn.Dropout(0.25)
        
        # self.conv6 = nn.Conv2d(64, 128, 3,padding=1)
        # self.bnorm5 = nn.BatchNorm2d(128)
        # self.pool4 = nn.MaxPool2d(2, 2)
        # self.drop3 = nn.Dropout(0.25)

        self.avgpl1 = nn.AvgPool2d(3) #32 x 5 x 5 ? 1 x 1 x 32

        self.fc1   = nn.Linear(32, 16)
        self.bnorm4 = nn.BatchNorm1d(16)
        self.drop2 = nn.Dropout(0.25)
        self.fc2   = nn.Linear(16, 10)


    def forward(self, x):
        x = self.pool1(self.bnorm1(F.relu(self.conv1(x))))
        x = self.pool2(self.bnorm2(F.relu(self.conv2(x))))
        x = self.bnorm3(F.relu(self.conv3(x)))
        x = self.drop1(x)
        # x = self.bnorm4(F.relu(self.conv4(x)))
        # x = self.pool2(x)
        # x = self.bnorm5(F.relu(self.conv5(x)))
        # x = self.pool3(x)
        # x = self.drop2(x)
        # x = self.bnorm5(F.relu(self.conv6(x)))
        # x = self.drop3(self.pool4(x))
        x = self.avgpl1(x)
        # x = F.relu(self.conv7(x))
        # x = x.view(-1, 10)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.drop2(self.bnorm4(x))
        x = self.fc2(x)
        return F.log_softmax(x)

In [ ]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
       BatchNorm2d-2           [-1, 32, 28, 28]              64
         MaxPool2d-3           [-1, 32, 14, 14]               0
            Conv2d-4           [-1, 32, 14, 14]           9,248
       BatchNorm2d-5           [-1, 32, 14, 14]              64
         MaxPool2d-6             [-1, 32, 7, 7]               0
            Conv2d-7             [-1, 32, 5, 5]           9,248
       BatchNorm2d-8             [-1, 32, 5, 5]              64
           Dropout-9             [-1, 32, 5, 5]               0
        AvgPool2d-10             [-1, 32, 1, 1]               0
           Linear-11                   [-1, 16]             528
      BatchNorm1d-12                   [-1, 16]              32
          Dropout-13                   [-1, 16]               0
           Linear-14                   

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:58: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [ ]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.5f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=0.09482090920209885 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 22.16it/s]



Test set: Average loss: 0.0820, Accuracy: 9791/10000 (97.91000%)



loss=0.15358248353004456 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 22.05it/s]



Test set: Average loss: 0.0536, Accuracy: 9838/10000 (98.38000%)



loss=0.14496706426143646 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 22.14it/s]


...


Test set: Average loss: 0.0367, Accuracy: 9901/10000 (99.01000%)



loss=0.012963290326297283 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 21.92it/s]



Test set: Average loss: 0.0351, Accuracy: 9909/10000 (99.09000%)



loss=0.011820237152278423 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 21.80it/s]



Test set: Average loss: 0.0353, Accuracy: 9909/10000 (99.09000%)

